<a href="https://colab.research.google.com/github/jairorodriguezarias/audio_analytics_lake/blob/main/audio_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Enable APIs

In [24]:
!gcloud config set project genai-387907

Updated property [core/project].


In [25]:
# !gcloud auth login
from google.colab import auth
auth.authenticate_user()

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=xzW3uoHLVFjGQPQmJ7LZ3rfZtGJImh&prompt=consent&access_type=offline&code_challenge=gssz1dTglSWNu-JrnSgCiVgKJmGIQ6ZTJ4Lyilj_M_w&code_challenge_method=S256

Enter authorization code: 4/0AfJohXmJCJP1NWJmZ8Oyj3KtrRogr_E4Ua8ZxzImmnR2IUPB6im0YU9JRG4WH4xhFKmEkQ

You are now logged in as [admin@jairorodriguez.altostrat.com].
Your current project is [genai-387907].  You can change this setting by running:
  $ gcloud config set p

In [26]:
!gcloud services enable cloudfunctions.googleapis.com --async
!gcloud services enable cloudbuild.googleapis.com --async
!gcloud services enable bigqueryconnection.googleapis.com --async
!gcloud services enable vision.googleapis.com --async

# Create BigLake connection

In [27]:
!bq mk --connection --location=us-east1 --project_id=genai-387907 --connection_type=CLOUD_RESOURCE biglake-connection

Bigquery service returned an invalid reply in mk operation: Error reported by server with missing
error fields. Server returned: {'error': {'code': 409, 'message': 'Already Exists: Connection
projects/899604603048/locations/us-east1/connections/biglake-connection', 'status':
'ALREADY_EXISTS'}}.

Please make sure you are using the latest version of the bq tool and try again. If this problem
persists, you may have encountered a bug in the bigquery client. Please file a bug report in our
public issue tracker:
https://issuetracker.google.com/issues/new?component=187149&template=0
Please include a brief description of the steps that led to this issue, as well as any rows that can
be made public from the following information:

== Platform ==
  CPython:3.10.12:Linux-5.15.120+-x86_64-with-glibc2.35
== bq version ==
  2.0.98
== Command line ==
  ['/tools/google-cloud-sdk/platform/bq/bq.py', '--application_default_credential_file', '/content/.config/legacy_credentials/admin@jairorodriguez.altos

**Show detail connection**


In [28]:
!bq show --connection 899604603048.us-east1.biglake-connection

Connection 899604603048.us-east1.biglake-connection

                    name                     friendlyName   description    Last modified         type        hasCredential                                            properties                                            
 ------------------------------------------ -------------- ------------- ----------------- ---------------- --------------- ----------------------------------------------------------------------------------------------- 
  899604603048.us-east1.biglake-connection                                23 Oct 17:18:38   CLOUD_RESOURCE   False           {"serviceAccountId": "bqcx-899604603048-dw2f@gcp-sa-bigquery-condel.iam.gserviceaccount.com"}  



**Grant access to SQ**

In [29]:
!gsutil iam ch serviceAccount:bqcx-899604603048-dw2f@gcp-sa-bigquery-condel.iam.gserviceaccount.com:objectViewer gs://vector_store-genai-387907-images

No changes made to gs://vector_store-genai-387907-images/


# Create table

In [16]:
!bq mk -d --data_location=us-east1 bq_images_pre

Dataset 'genai-387907:bq_images_pre' successfully created.


In [32]:
%%bigquery results --project genai-387907
SELECT * FROM `genai-387907.bq_images_pre.images`

Could not save output to variable 'results'.

ERROR:
 ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7a034b181750>)


In [38]:
#Replace 'project_id' with your BigQuery project ID

from google.cloud import bigquery
client = bigquery.Client(project='genai-387907')


In [44]:
#Create external table

sql_query = ('''CREATE OR REPLACE EXTERNAL TABLE
              `genai-387907.bq_images_pre.images`
            WITH CONNECTION `us-east1.biglake-connection` OPTIONS (
                object_metadata="DIRECTORY",
                uris = ['gs://vector_store-genai-387907-images/*' ],
                max_staleness=INTERVAL 30 MINUTE,
                metadata_cache_mode="AUTOMATIC");''')

df = client.query(sql_query)

In [52]:
# Read table
sql_query = ('''select * from
              `bq_images_pre.images`;''')

df = client.query(sql_query).to_dataframe()
df

,uri,generation,content_type,size,md5_hash,updated,metadata
0,gs://vector_store-genai-387907-images/00000001...,1697613110853651,image/jpeg,248239,1bc0990f4f1e1242585f5f0484c7bad7,2023-10-18 07:11:50.892000+00:00,[]
1,gs://vector_store-genai-387907-images/00773070...,1697613110755187,image/jpeg,481144,32255f60bc5217143de55f1cd87c131d,2023-10-18 07:11:50.796000+00:00,[]
2,gs://vector_store-genai-387907-images/00854548...,1697613111155008,image/jpeg,352723,8c73835ce0189b514b1aa3fe4796d620,2023-10-18 07:11:51.193000+00:00,[]
3,gs://vector_store-genai-387907-images/02644743...,1697613112459384,image/jpeg,886010,dca8043421773cd545a76b346306b7b4,2023-10-18 07:11:52.499000+00:00,[]
4,gs://vector_store-genai-387907-images/03100099...,1697613111952518,image/jpeg,81761,23518325ad651adb1bd086acddf92d98,2023-10-18 07:11:51.992000+00:00,[]
...,...,...,...,...,...,...,...
104,gs://vector_store-genai-387907-images/90000001...,1697613152659850,image/jpeg,1001427,a5e3e5aac385b9e38ddb76fec219fe27,2023-10-18 07:12:32.699000+00:00,[]
105,gs://vector_store-genai-387907-images/96216409...,1697613153239526,image/jpeg,318260,416159ca1cd5c46c5aa89cfc0541f645,2023-10-18 07:12:33.280000+00:00,[]
106,gs://vector_store-genai-387907-images/98781663...,1697613153568094,image/jpeg,623341,cc6353ee21d43fbf65ab01206d4e311f,2023-10-18 07:12:33.606000+00:00,[]
107,gs://vector_store-genai-387907-images/98781778...,1697613153756363,image/jpeg,230187,26c86e620bb1f85103952ebe8c012af6,2023-10-18 07:12:33.794000+00:00,[]


# Cloud function

Save this in two files:
main.py
requirements.txt

**main.py**

```
#
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

from google.protobuf.json_format import MessageToJson
from google.cloud import vision
import json

def cloud_ai_vision(request):
    try:
        client = vision.ImageAnnotatorClient()
        request_json = request.get_json()
    
        calls = request_json['calls']
        return_value = []
        for call in calls:
            image = vision.Image(
                source=vision.ImageSource(image_uri=call[0]))
            response = client.object_localization(image=image)  
            serialized_proto_plus = vision.AnnotateImageResponse.serialize(response)
            response = vision.AnnotateImageResponse.deserialize(serialized_proto_plus)
            response_json = MessageToJson(response._pb)
            return_value.append(str(response_json))
            return_json = json.dumps({"replies": return_value})
        return return_json
    except Exception as e:
        return str(e.message)

```

**requirements.txt**

```
functions-framework==3.2.0
protobuf
google-cloud-vision
google-cloud-logging
```

**deploy function**

```
gcloud functions deploy cloud_ai_vision \
--runtime python310 \
--trigger-http \
--ingress-settings all \
--region us-east1 \
--source ./function/
```